In [ ]:
import pandas as pd

try:
    crm_df = pd.read_excel("CRM.xlsx")
    voucher_df = pd.read_excel("voucher_report-JFM_25.xlsx")
    account_df = pd.read_excel("account_JFM_25.xlsx")

    #crm_filtered = crm_df[crm_df['voucher_amount'].notna() & (crm_df['voucher_amount'] != '')]
    crm_filtered = crm_df[
    crm_df['voucher_amount'].notna() & (crm_df['voucher_amount'] != '')
    ].drop_duplicates(subset='reference_number')

    crm_grouped = crm_filtered.groupby('reference_number', as_index=False)['voucher_amount'].sum()
    voucher_grouped = voucher_df.groupby('reference_number', as_index=False)['total'].sum()
    account_grouped = account_df.groupby('Ref. No', as_index=False)['Bill amount'].sum()

    m1_df = voucher_grouped.merge(account_grouped, left_on='reference_number', right_on='Ref. No', how='inner')
    m2_df = m1_df.merge(crm_grouped, on='reference_number', how='inner')

    mismatch = m2_df[round(m2_df['Bill amount'], 2) != round(m2_df['voucher_amount'], 2)]

    mismatch['Remarks'] = 'billing amount mismatch'
    mismatch.to_excel("billing_mismatch_report.xlsx", index=False)

    print("✅ 'billing_mismatch_report.xlsx' created for billing amount mismatches.")

except Exception as e:
    print(f"❌ Error: {e}")
